# Primera práctica de TGINE: TF-IDF con una subred de Reddit

## La Subred elegida debido a su gran potencial de texto para generar un buen corpus ha sido la de 'lotr' (Lord of the Rings) ya que contaba con muchos posts y comentarios.

### La primera parte de la práctica consistía en leer de dicha subred los post y comentarios que después serían los documentos que le pasaríamos posteriormente al vectorizador, escogiendo para ellos los más recientes y los más populares. Pero para no estar llamando a la API de Reddit en cada ejecución se opta por hacer un primer script de Python que guarde la información obtenida en un fichero de datos semiestructurados como es XML en disco, y después en un segundo script leer del mismo sin necesidad de llamar a la API.

### Para ello se define el siguiente XML Schema, que luego nos servirá para validar que el documento está bien estructurado antes de guardarlo en disco.

- esquema-reddit.xsd

- Como se puede apreciar el XML esquema sigue un orden jeráquico de los elementos que nos vamos a encontrar en la subred. El elemento 'subreddit' sería el elemento raíz que recoge un atributo nombre para la red, en este caso será 'lotr'. Después tendrá dos hijos, denominados 'apartados' que recogerán los post más recientes por una parte y los más populares por otra. En cada uno de los apartados estarán los posts correspondientes, y dentro de los posts sus comentarios. A efectos de la lógica queda de este modo bien definida la jerarquía de la subred. No obstante, para el vectorizador tanto posts como comentarios serán documentos individuales.

## Primer Script: practicaRedditCreacionXML.py 

In [ ]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Mon Nov 28 22:48:20 2016

@author: moises
"""
#Importación de librerías
import sys
import praw
import datetime
import xml.etree.ElementTree as ET
from lxml import etree
from pprint import pprint

#Definición y cuerpo de las funciones
def conectarAReddit():
    UA = 'linux:practicaReddit.py:v1.0 (by /u/Zollkron)'
    wrapper = praw.Reddit(UA)
    wrapper.set_oauth_app_info(client_id='Nmmtds24J7nFhw',
                      client_secret='ebYaWc_8lSs1mghubwbGvCWDYvw',
                      redirect_uri='http://127.0.0.1:65010/'
                                   'authorize_callback')
    return wrapper

### Tal y como se pide en el enunciado se crea una primera función para conectar a Reddit usando la librería praw y usando la autenticación oauth en lugar de la ya desfasada autenticación mediante usuario y contraseña.

#### NOTA: Se usa pprint para la identificación de los atributos de los objetos devueltos por reddit ya que en la documentación  no he podido encontrar nada que me lo indicase. Se deja como comentario para que no interfiera en el la ejecución del script, pero me gustaría dejarlo constar.

In [ ]:
def obtenerSubReddit(wrapper, nombre):
    return wrapper.get_subreddit(nombre)

### Tras esto se crea una función para desde el wrapper de praw obtener la subred indicada por parámetro, en nuestro caso 'lotr'.

In [ ]:
def imprimirSubmissions(submissions):
    for submission in submissions:
        print("-==Post==-")
        print("")
        creation_date = datetime.datetime.fromtimestamp(int(submission.created))
        print(creation_date)
        print("")
        print(submission.title)
        print("")
        print(submission.selftext)
        print("")
        print(submission.author)
        print("")
        #pprint(vars(submission))
        for comment in submission.comments:
            print("-==Comment==-")
            print("")
            creation_date = datetime.datetime.fromtimestamp(int(comment.created))
            print(creation_date)
            print("")
            print(comment.body)
            print("")
            print(comment.author)
            print("")
            #pprint(vars(comment))

### He creado una función auxiliar denominada imprimirSubmissions que muestra por consola los posts así como sus comentarios para comprobar en primera instancia que el wrapper ha funcionado. 

In [ ]:
def construirXML(subreddit):
    xmlnsUri = "practicaReddit"
    xmlnsXsi = "http://www.w3.org/2001/XMLSchema-instance"
    xsiSchema = "practicaReddit practicaReddit.xsd"
    raiz = ET.Element("subreddit", nombre=subreddit.display_name, xmlns=xmlnsUri, 
                  **{'xmlns:xsi':xmlnsXsi,'xsi:schemaLocation':xsiSchema})
    #Se crea un elemento apartado para los posts más recientes dentro del XML
    postRecientes = ET.SubElement(raiz, "apartado", nombre="Posts Mas Recientes")
    #Se obtienen los posts junto con sus comentarios a través del objeto subreddit obtenido mediante el wrapper
    submissions = subreddit.get_hot() #Ultimos posts
    #Para cada Post
    for submission in submissions:
        #Obtenemos su fecha de creación
        creation_date = datetime.datetime.fromtimestamp(int(submission.created))
        #La formateamos de modo que nos la acepte el tipo datetime de XML
        fecha_formateada = creation_date.strftime("%Y-%m-%dT%H:%M:%S")
        #Y creamos un elemento post dentro del documento XML en memoria
        post = ET.SubElement(postRecientes, "post", titulo=submission.title , \
                             autor=submission.author.name,fecha=fecha_formateada,texto=submission.selftext)
        #Para cada comentario dentro del Post
        for comment in submission.comments:
            #Obtenemos su fecha de creación
            creation_date = datetime.datetime.fromtimestamp(int(comment.created))
            #La formateamos de modo que nos la acepte el tipo datetime de XML
            fecha_formateada = creation_date.strftime("%Y-%m-%dT%H:%M:%S")
            #Y creamos un elemento comentario dentro del documento XML en memoria
            ET.SubElement(post, "comentario", autor=comment.author.name, \
                          fecha=fecha_formateada, texto=comment.body)
    #Se repite el mismo proceso para los posts más populares
    postPopulares = ET.SubElement(raiz, "apartado", nombre="Posts Mas Populares")
    submissions = subreddit.get_top() #Post Mas Populares
    for submission in submissions:
        creation_date = datetime.datetime.fromtimestamp(int(submission.created))
        fecha_formateada = creation_date.strftime("%Y-%m-%dT%H:%M:%S")
        post = ET.SubElement(postPopulares, "post", titulo=submission.title, \
                      autor=submission.author.name, fecha=fecha_formateada, \
                      texto=submission.selftext)
        for comment in submission.comments:
            creation_date = datetime.datetime.fromtimestamp(int(comment.created))
            fecha_formateada = creation_date.strftime("%Y-%m-%dT%H:%M:%S")
            ET.SubElement(post, "comentario", autor=comment.author.name, \
                          fecha=fecha_formateada, texto=comment.body)
    #Por último devolvemos el elemento raíz del documento XML
    return raiz

### La siguiente función, construirXML, es el corazón de este primer script, y es el que se encarga de construir el XML de toda la subred obtenido primero en memoria.

In [ ]:
def validarXML(xml):
   fichero = open("esquema-reddit.xsd","r")
   schema = etree.XML(fichero.read())
   xmlSchema = etree.XMLSchema(schema)
   xmlParser = etree.XMLParser(schema = xmlSchema)
   try:
       etree.fromstring(ET.tostring(xml), xmlParser) 
   except etree.XMLSyntaxError:
       mensaje = "El XML no se ha validado porque:\n" + str(sys.exc_info()[1])
       print(mensaje)
       return False     
   else:
       mensaje = "El XML generado es correcto."
       print(mensaje)
       return True    

### Tras ello hay una función de validarXML para asegurarnos que el XML construido en memoria es correcto según el esquema que hemos definido antes de persistirlo en disco. 

In [ ]:
#Principio de la identación de Python en este script (que coloquialmente podemos asimilar a la función main)
wrapper = conectarAReddit()
subreddit = obtenerSubReddit(wrapper, 'lotr')
#pprint(vars(subreddit))
print(subreddit.display_name)
#submissions = subreddit.get_hot() #Ultimos posts
#imprimirSubmissions(submissions)

xml = construirXML(subreddit)
resultado = validarXML(xml)
if resultado == True:
    print("Guardamos")
    from xml.dom import minidom
    xmlstr = minidom.parseString(ET.tostring(xml)).toprettyxml(indent="   ")
    with open("LordOfTheRings.xml", "w") as f:
        f.write(xmlstr.encode('utf-8'))

### Lo que viene a continuación se podría decir que es "la función main" del script, pero para Python en realidad es el principio de su identación en este script. Tras generar el XML ya tenemos los datos necesarios para leerlo, extraer su corpus y pasaro por el vectorizador de scikit-learn, y eso es justo lo que hará el segundo script. 

## Segundo Script: practicaRedditVectorizacion.py

In [ ]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Tue Dec  6 18:05:54 2016

@author: moises
"""
#Importación de librerías
import operator
from lxml import etree
from sklearn.feature_extraction.text import TfidfVectorizer

#Definición y cuerpo de las funciones
def leerXML(ruta):
    return etree.parse(ruta)

### La primera función sólo se limita a leer el contenido del fichero XML guardado en disco por el anterior script, en nuestro caso particular es 'LordOfTheRings.xml'.

In [ ]:
def extraerCorpus(raiz):
    corpus = []
    for apartado in raiz:
        for post in apartado:
            #print post.get('texto')
            corpus.append(post.get('texto'))
            for comentario in post:
                #print comentario.get('texto')
                corpus.append(comentario.get('texto'))
    return corpus

### La segunda función extraerCorpus es para que construya una colección de documentos de texto tanto de los posts como de los comentarios, cada uno formando un documento detexto individual.

In [ ]:
#Principio de la identación de Python en este script (que coloquialmente podemos asimilar a la función main)

#Leemos el documento XML creado con el script anterior
documentoXML = leerXML("LordOfTheRings.xml")
#Extraemos el elemento raíz
raiz = documentoXML.getroot()
#A partir de dicho elemento raíz extraemos el Corpus del texto
corpus = extraerCorpus(raiz)
#print corpus

### Leemos el fichero XML guardado en disco por el anterior script, en nuestro caso particular es 'LordOfTheRings.xml'. Tras ello, obtenemos el elemento raíz correspondiente con la subred 'lotr'. A continuación, le pasamos el elemento raíz a la función extraerCorpus para que construya una colección de documentos de texto. Esta colección de documentos de texto, a la que llamamos Corpus, es lo que le pasaremos al vectorizador.

In [ ]:
#Creamos el vectorizador para que analice por palabra, y que filtre todas aquellas
#palabras que aparezcan en menos de 10 documentos así como las stop_words en inglés
vectorizer = TfidfVectorizer(analyzer='word', min_df = 10, stop_words = 'english')
#Usando el vectorizador extraemos la matriz vectorizada de todas las palabras
matrizVectorizada = vectorizer.fit_transform(corpus)
#Usamos también el vectorizador para extraer todas las palabras que haya encontrado
palabras = vectorizer.get_feature_names()
#print palabras
#print matrizVectorizada

### A continuación, nos creamos un vectorizador del tipo TfidfVectorizer indicándole que lo que queremos es analizar palabras, y que filtre todas aquellas que aparezcan en menos de 10 documentos y las stop-words en inglés. Una vez preparado el vectorizador, lo usamos para obtener la matriz vectorizada a partir de nuestro corpus, de paso obtenemos un listado de todas las palabras que ha encontrado que nos será útil después.

In [ ]:
#Como la matriz vectorizada que nos devuelve el vectorizador está comprimida
#para ahorrar espacio, obtenemos la matriz de densidad para poder acceder mejor 
#a los datos
matrizDensa = matrizVectorizada.todense()

### La matriz vectorizada que nos devuelve el vectorizador está comprimida para ahorrar espacio, por ello si queremos tratar los datos que contiene es mejor obtener su matriz densa. En dicha matriz ya tenemos todo lo que necesitamos: las puntuaciones TF-IDF de las palabras de todo el corpus.

In [ ]:
#Construimos un mapa para guardar todas las palabras del corpus
#así como su puntuación TF-IDF. De momento la inicializamos a 0 de puntuación.
mapaPalabras = {palabra.encode("utf-8"):float(0) for palabra in palabras}

### Nos creamos a partir del listado de palabras encontradas en el corpus por el vectorizador un mapa(clave,valor) en el que la clave será la palabra en cuestión (codificada en 'utf-8'), y el valor la suma acumulada TF-IDF que nos encontremos al recorrer la matriz densa. Al principio inicializamos todos los valores del mapa a 0.

In [ ]:
idDocumento = 0
#Empezamos a recorrer los documentos
for doc in matrizDensa:
    #print "Documento %d" %(idDocumento)
    idPalabra = 0
    lista = doc.tolist()
    #Nos aseguramos de recorrer todas las filas de la matriz del documento
    for i in range(len(lista)):
        #Recorremos las palabras de la fila actual (i)
        for puntuacionTfIdf in doc.tolist()[i]:
            #Para cada palabra comprobamos si su tf-idf es mayor que 0
            if puntuacionTfIdf > 0:
                palabra = palabras[idPalabra]
                #Sumamos la puntuación TF-IDF de este documento a la palabra
                mapaPalabras[palabra.encode("utf-8")] += float(puntuacionTfIdf)
            idPalabra +=1
    idDocumento +=1

### Tras ello recorremos la matriz densa, primero documento a documento, después fila a fila de cada submatriz del documento, y después ya sí por fin, palabra por palabra. Si vemos que la palabra que estamos observando tiene un valor de TF-IDF superior a 0, lo acumulamos en su acumulador correspondiente dentro del mapa.

In [ ]:
#Llegado a este punto ya tenemos todas las palabras con su puntuación TF-IDF total
#respecto a TODO el corpus. Tras esto tenemos que ordenarlas de mayor a menor
#puntuación IF-IDF y quedarnos con las 10 primeras
palabrasCentrales = sorted(mapaPalabras.items(), key=operator.itemgetter(1), reverse=True)[:10]

### Una vez terminada de recorrer toda la matriz densa ya tenemos todas las puntuaciones TF-IDF acumuladas por palabras en el mapa, pero el mapa está desordenado y sólo nos interesa las primeras 10 palabras centrales, que son las que mayor suma acumulada de TF-IDF tienen. Y justo eso es lo que hacemos al final, ordenamos el mapa por valor de forma decreciente (de mayor a menor puntuación TF-IDF) y le decimos que nos quedamos con los 10 primeros.

In [ ]:
#Tras quedarnos con las palabras centrales las mostramos así como su puntuación
print('{0: <10} {1}'.format("Palabra", "TF-IDF"))
print "-------  ----------------"
for palabra, puntuacionTfIdf in palabrasCentrales:
    print('{0: <10} {1}'.format(palabra, puntuacionTfIdf))

###  Tras ello mostramos finalmente por consola y de forma formateada las 10 palabras centrales. Siendo este el resultado final en mi caso en el momento de probarlo:

- Palabra    TF-IDF
- -------  ----------------
- like       13.1177156153
- just       12.6630773332
- great      11.8270984841
- com        11.0187180747
- movie      10.2632426618
- did        9.48731187443
- love       8.25114118716
- work       8.11053026027
- sauron     7.76398522288
- fellowship 7.50957198986

## Conclusión 

### A la vista del resultado obtenido podemos observar como dentro de los términos más usados en el corpus según su puntuación TF-IDF se expresan sentimientos como 'like', 'great' o 'love' lo que indica un cierto amor de los fans por el mundo de El Señor de los Anillos. También se nombra la película 'movie' o el trabajo 'work' (obra) refiriéndose a los libros. Se nombra al antagonista 'sauron' quien es en realidad El Señor de los Anillos que los controla todos con el anillo único. Y por último sale la palabra 'fellowship' que es la compañía que se forma para destruirlo. También se 'cuelan' términos que a priori no tienen nada que ver con el mundo de ESDLA, como 'just', 'com', o 'did' que pueden haber salido por resultar expresiones muy comunes en inglés, y en el caso de 'com' podría ser que se refiriese a un apócope de 'company' (compañía) que ya no queda claro si es para referirse a la compañía del anillo, a la que el autor del libro denomina 'fellowship', o si se trata de la compañía que hace la película, o algún otro tipo de compañía que se nombra en esa subred. La cuestión es que la técnica TF-IDF ha sido bastante acertada con la mayoría de los términos que identifican al mundo de el ESDLA, ya que son realmente significativos por lo menos en lo que a los datos extraidos de esa subred se refiere.